# Towards new features

Unfortunately, parameter tuning for both Random Forests and SVM seems not to lead to large increases in classification performance. We might be able to increase, however, by extracting new features. Currently, the similarity score is calculated as in the paper. This similarity score has a number of oddities:
- the presence of a k-gram in a model is weighted equally, regardless of k. Each k-gram leads to two (k-1)-grams. This means that k-grams with higher k are assigned lower weights.
- The equal weighting does not take into account whether a word is somewhat specific to one of the language distributions. In the paper, they show for example that "prove that" is associated with Russians and "refute" with the Dutch. The weighting scheme does not take such correlations into account. We could take them into account by constructing a language distribution per language and calculating the idf-tf per word present.

Even though it has problems, one should note that the highest classification performance in T1-language classification is typically reached by analyzing the n-grams directly, rather than similarity scores to n-gram models. Accuracies in the range of 80-90% have been reported for this task, which is conceptually more difficult than native-language classification as it concerns classification of the native language of the writer him/herself (e.g. Jarvis/Bestgen/Pepper, Gebre/Zampierie/Wittenburg). Some interesting results:
- In Groningen, character n-grams in the range of 8-10 alone led to very high classification accuracies. We might want to see if we can use these.
- Somewhat unexpectedly, an ensemble of learners applied to different types of n-grams has been reported to perform better than a single learner applied to the same n-grams by itself.

## Load the necessary stuff

In [1]:
# Import necessary packages
import pandas as pd
import time
import datetime
import numpy as np
import re
import sys
import random
import math
from nltk import FreqDist, ngrams, sent_tokenize, word_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn import svm
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold, ParameterGrid, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
        
def load_object(filename):
    with open(filename, 'rb') as f:
        x = pickle.load(f)
    return(x)

# Load the training data. Scramble the rows (sometimes this is important for training)
df = pd.read_csv("python_data/train",sep="\t",error_bad_lines=False,encoding="utf-8")
df = df.sample(frac=1, random_state = 54021)
df['native'] = np.where(df['native_lang']=='EN', "native", "non-native")

# Load the training data. Downsample non-English such that it is balanced.
print("Loading the training and validation data...")
training = pd.concat([df[df.native == "non-native"].sample(sum(df.native == "native"), random_state = 1810), df[df.native=="native"]])
training = training.sample(frac=1, random_state = 1318910)
training.native = training.native.astype('category')

# Load the validation data. Again, downsample such that it is balanced.
validation = pd.read_csv("python_data/development",sep="\t",error_bad_lines=False,encoding="utf-8")
validation['native'] = np.where(validation['native_lang']=='EN', "native", "non-native")
validation = pd.concat([validation[validation.native == "non-native"].sample(sum(validation.native == "native"), random_state = 1), validation[validation.native=="native"]])
validation.native = validation.native.astype('category')

Loading the training and validation data...


## Incorporating tf idf

We want to construct a tf-idf library for all languages separately. Hence, first derive a language distribution for all 20 languages.

In [3]:
def sum_keys(d):
    return (0 if not isinstance(d, dict) else len(d) + sum(sum_keys(v) for v in d.values()))

def pruned_language_distribution(n, m, lowerfreqlimit, training, LANGUAGES):
    """Calculate the word n grams distribution up to n, the character n gram distribution up to m.
    @n: consider k-grams up to and including n for words, part of speech tags and word sizes.
    @m: consider k-grams up to and including m for characters. We assume m >= n.
    @lowerfreqlimit: number below which we consider words misspellings, odd words out or unique.
    @training: training data to retrieve the language distribution from.
    @LANGUAGES: languages based on which we classify.
    """
    
    language_dist = {}

    for language in LANGUAGES:
        language_dist[language] = {"words": dict(zip(range(1, n+1), [FreqDist() for i in range(1, n+1)])),
                               "tags": dict(zip(range(1, n+1), [FreqDist() for i in range(1, n+1)])),
                               "chars": dict(zip(range(1, m+1), [FreqDist() for i in range(1, m+1)])),
                               "w_sizes": dict(zip(range(1, n+1), [FreqDist() for i in range(1, n+1)]))}
        
    # Iterate first over k. This is required as we need to know the full k-1 distributions to see if we should add a 
    # k-gram to the dictionary.
    kmax = 0
    for k in range(1, n+1):
        for language, text, struc in training.itertuples(index=False):
            
            for sentence in sent_tokenize(text):
                
                # Get the necessary input structures for the ngrams-function. It is sentence for "chars".
                token=word_tokenize(sentence) 
                wordlens = [len(word) for word in token]
                
                # Note, for any gram, there exist 2 subgrams of all but the first and all of the last element. Let us
                # only update the dictionary if the total count of these subgrams exceeds the lower limit. This prevents
                # an unnecessary combinatorial explosion.
                for gram in ngrams(sentence,k):
                    if k == 1: 
                        language_dist[language]["chars"][k][gram] += 1
                    elif language_dist[language]["chars"][k-1].get(gram[1:],0)+language_dist[language]["chars"][k-1].get(gram[:-1],0) > 2*lowerfreqlimit:
                        language_dist[language]["chars"][k][gram] += 1
                        
                for gram in ngrams(token,k):
                    if k == 1:
                        language_dist[language]["words"][k][gram] += 1
                    elif language_dist[language]["words"][k-1].get(gram[1:],0)+language_dist[language]["words"][k-1].get(gram[:-1],0) > 2*lowerfreqlimit:
                        language_dist[language]["words"][k][gram] += 1
                        
                for gram in ngrams(wordlens,k):
                    if k == 1:
                        language_dist[language]["w_sizes"][k][gram] += 1
                    elif language_dist[language]["w_sizes"][k-1].get(gram[1:],0)+language_dist[language]["w_sizes"][k-1].get(gram[:-1],0) > 2*lowerfreqlimit:
                        language_dist[language]["w_sizes"][k][gram] += 1
                        
            # Now for the tokenized structures (tags)
            for sentence in sent_tokenize(struc):
                token=word_tokenize(sentence)
                for gram in ngrams(token,k):
                    if k == 1:
                        language_dist[language]["tags"][k][gram] += 1
                    elif language_dist[language]["tags"][k-1].get(gram[1:],0)+language_dist[language]["tags"][k-1].get(gram[:-1],0) > 2*lowerfreqlimit:
                        language_dist[language]["tags"][k][gram] += 1
                        
    # Also construct it for higher order k-grams for characters.
    for k in range(n+1, m+1):
        for language, tokenized_sents, tokenized_struc in training.itertuples(index=False):
            for sentence in tokenized_sents:
                for gram in ngrams(sentence,k):
                    if language_dist[language]["chars"][k-1].get(gram[1:],0)+language_dist[language]["chars"][k-1].get(gram[:-1],0) > 2*lowerfreqlimit:
                        language_dist[language]["chars"][k][gram] += 1
                           
    return language_dist

def construct_lodds_ratio_dict(lang_dis):
    
    n = len(lang_dis['EN']['words'])
    m = len(lang_dis['EN']['chars'])
    
    """ 
    In this function, we want to compute the odds ratio for each of the n-grams, and return a dictionary with these values.
    For each non-English language, we will add a pseudocount of .5 to prevent infinity.
    """
    for lang in lang_dis.keys():
        for gramtype in lang_dis[lang].keys():
            for k in lang_dis[lang][gramtype].keys():
                b = sum_keys(lang_dis[lang][gramtype][k])
                d = sum_keys(lang_dis['EN'][gramtype][k])
                for key in lang_dis[lang][gramtype][k].keys():
                    a = lang_dis[lang][gramtype][k].get(key,0.5)
                    c = lang_dis['EN'][gramtype][k].get(key,0.5)
                    lang_dis[lang][gramtype][k][key] = math.log((a/b)/(c/d))

                # Release the memory.
                lang_dis[gramtype][k].clear()
    return(lang_dis)

We first need to construct a language distribution for different languages. Let us not do this by downsampling to the minimum number, but rather take the same training data as previously. The log-odds ratio takes into account imbalance in class.

In [ ]:
# Derive the language distribution from the training data.
print("Deriving the language distribution from training data...")
start = time.time()
lang_dis = pruned_language_distribution(4,4,10,training[['native','text_clean','text_structure']], training.native_lang.unique())
end = time.time()
print("Language distribution constructed in {} seconds".format(end-start))

# Save it and clear it to save memory.
save_object(lang_dis,"trained_lang_dis_20_lang_ll10")
lang_dis.clear()

Now we can calculate the log odds ratios per entry

In [ ]:
lodds_ratio = construct_lodds_ratio_dict(lang_dis)